In [3]:
import pandas as pd 
import numpy as np

from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
# import torchvision
# import torchvision.transforms as transforms


from datetime import datetime
from osgeo import gdal
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize



from keras.layers import Flatten, Dense
import dateutil
import glob

In [4]:
train_labels = pd.read_csv("train_labels.csv")
grid_metadata = pd.read_csv("grid_metadata.csv")
satellite_metadata = pd.read_csv("satellite_metadata.csv")
satellite_metadata['Date'] =  pd.to_datetime(satellite_metadata['time_end'], format='%Y-%m-%d')

print(train_labels)


                   datetime grid_id       value
0      2018-02-01T08:00:00Z   3S31A   11.400000
1      2018-02-01T08:00:00Z   A2FBI   17.000000
2      2018-02-01T08:00:00Z   DJN0F   11.100000
3      2018-02-01T08:00:00Z   E5P9N   22.100000
4      2018-02-01T08:00:00Z   FRITQ   29.800000
...                     ...     ...         ...
34307  2020-12-31T18:30:00Z   P8JA5  368.611111
34308  2020-12-31T18:30:00Z   PW0JT  294.425000
34309  2020-12-31T18:30:00Z   VXNN3  224.857143
34310  2020-12-31T18:30:00Z   VYH7U  287.000000
34311  2020-12-31T18:30:00Z   ZF3ZW  410.500000

[34312 rows x 3 columns]


In [5]:
# train_labels = train_labels.sample(1000, random_state=42)
train_labels = train_labels.sample(3000, random_state=42)

In [6]:

def get_grid_data(metadata, grid_id):
    return metadata[metadata["grid_id"] == grid_id]

def fetch_satellite_meta(metadata, datetime, location, datatype, split):
    if location == "Delhi":
        location = "dl"
    elif location == "Taipei":
        location = "tpe"
    else:
        location = "la"

    metadata = metadata[metadata['location'] == location]
    metadata = metadata[metadata['product'] == datatype]
    metadata = metadata[metadata['split'] == split]
    dateobject = dateutil.parser.parse(datetime)
    return metadata.loc[(metadata['Date'].dt.month == dateobject.month) & 
                        (metadata['Date'].dt.day == dateobject.day) &
                        (metadata['Date'].dt.year <= dateobject.year)]

In [7]:


# Opens the HDF file
def load_data(FILEPATH):
    ds = gdal.Open(FILEPATH)
    return ds

def fetch_subset(granule_id):
    ds = load_data("dataset/" + granule_id)
    ds.GetSubDatasets()[0]
    raster = gdal.Open(ds.GetSubDatasets()[8][0]) #grid5km:cosSZA features only
    band = raster.GetRasterBand(1)
    band_arr = band.ReadAsArray()
    return band_arr


def fetch_training_features(grid_id, datetime, split):
    temp = get_grid_data(grid_metadata, grid_id)
    sat_met = fetch_satellite_meta(satellite_metadata, 
                               datetime, 
                               temp.iloc[0]['location'], 
                               "maiac", 
                               split)
    counter = 0
    features = None
    for i in range(len(sat_met)):
        counter+=1
        granule_id = sat_met.iloc[i]['granule_id']
        subset = fetch_subset(granule_id)
        if features is None:
            features = subset
        else:
            features+=subset
    return features/counter

def generate_features(train_labels, split):
    labels = []
    features = []
    for i in range(len(train_labels)):
        if i % 500 == 0: print(i)
        feature = fetch_training_features(train_labels.iloc[i]['grid_id'], train_labels.iloc[i]['datetime'], split)
        features.append(np.array(feature).reshape(-1))
        if split == "train":
            labels.append(train_labels.iloc[i]['value'])
    return np.array(features), np.array(labels)



In [8]:

features, labels = generate_features(train_labels, "train")

0
500
1000
1500
2000
2500


In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, labels,  test_size=0.30, random_state=42)

shape_tuple = (240,240)
X_train_reshape = []
for i in range(X_train.shape[0]):
    X_train_reshape.append(X_train[i].reshape(shape_tuple))
X_train_reshape = np.stack(X_train_reshape, axis=0 )
print(X_train_reshape.shape)

X_test_reshape = []
for i in range(X_test.shape[0]):
    X_test_reshape.append(X_test[i].reshape(shape_tuple))
X_test_reshape = np.stack(X_test_reshape, axis=0 )
print(X_test_reshape.shape)


(2100, 240, 240)
(900, 240, 240)


In [10]:

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)


for i in range(X_train.shape[0]):
    X_train_reshape[i] = normalize(X_train_reshape[i])
    
for i in range(X_test.shape[0]):
    X_test_reshape[i] = normalize(X_test_reshape[i])

In [11]:
X_train = torch.from_numpy(X_train_reshape)
X_test = torch.from_numpy(X_test_reshape)
y_train = torch.from_numpy(y_train)
y_train = torch.unsqueeze(y_train,dim=1)
y_test = torch.from_numpy(y_test)
y_test = torch.unsqueeze(y_test,dim=1)


In [12]:

input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
mse_loss = nn.MSELoss()
output = mse_loss(input, target)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)

input:  tensor([[-0.7204, -1.1817, -2.3289, -0.6339,  0.3079],
        [ 0.9281, -1.2470, -0.4750,  0.0217,  0.3422],
        [ 0.3498,  0.1529, -1.3565,  0.2168,  0.5467]], requires_grad=True)
target:  tensor([[ 0.9591,  2.1842, -0.6572, -0.7069,  0.1278],
        [ 0.2686, -0.1737,  0.6260, -1.6006, -2.0905],
        [-1.5156, -2.4388,  0.3009,  1.7885,  0.1222]])
output:  tensor(2.9284, grad_fn=<MseLossBackward>)


In [13]:


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=3,kernel_size=5, padding=0) #3,6,5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 84)
        self.fc1 = nn.Linear(51984,84)
        self.fc2 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x


In [14]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 3, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=51984, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=1, bias=True)
)


In [15]:
batch_size = 4 #unused
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)


In [16]:
# for epoch in range(2):  # loop over the dataset multiple times
    # running_loss = 0.0
epoch = 1
running_loss = 0
for i in range(X_train.shape[0]):
    if i % 200 == 0: print(i)
    # data[i].unsqueeze(0)
    inputs = X_train[i].unsqueeze(0).unsqueeze(0)
    # print(X_train[i].unsqueeze(0).unsqueeze(0).shape)
    inputs = inputs.float()
    labels = y_train[i]
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        running_loss = 0.0
print('Finished Training')

0


C:\Users\16028\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


200
400
600
800
1000
1200
1400
1600
1800
[2,  2000] loss: 6178.205
2000
Finished Training


In [18]:
with torch.no_grad():
    net.eval()
    preds = []
    for i in range(X_test.shape[0]):
        preds.append(net(X_test[i].unsqueeze(0).unsqueeze(0).float()))
        